<a href="https://colab.research.google.com/github/sokrypton/ColabDesign/blob/v1.1.1/af/examples/afdesign_hotspot_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AfDesign - binder + hotspot test

In [ ]:
#@title setup
%%time
import os
if not os.path.isdir("params"):
  # get code
  os.system("pip -q install git+https://github.com/sokrypton/ColabDesign.git@v1.1.1")
  # for debugging
  os.system("ln -s /usr/local/lib/python3.*/dist-packages/colabdesign colabdesign")
  # download params
  os.system("mkdir params")
  os.system("apt-get install aria2 -qq")
  os.system("aria2c -q -x 16 https://storage.googleapis.com/alphafold/alphafold_params_2022-12-06.tar")
  os.system("tar -xf alphafold_params_2022-12-06.tar -C params")

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
from colabdesign import mk_afdesign_model, clear_mem
from IPython.display import HTML
from google.colab import files

def get_pdb(pdb_code=""):
  if pdb_code is None or pdb_code == "":
    upload_dict = files.upload()
    pdb_string = upload_dict[list(upload_dict.keys())[0]]
    with open("tmp.pdb","wb") as out: out.write(pdb_string)
    return "tmp.pdb"
  elif os.path.isfile(pdb_code):
    return pdb_code
  elif len(pdb_code) == 4:
    os.system(f"wget -qnc https://files.rcsb.org/view/{pdb_code}.pdb")
    return f"{pdb_code}.pdb"
  else:
    os.system(f"wget -qnc https://alphafold.ebi.ac.uk/files/AF-{pdb_code}-F1-model_v3.pdb")
    return f"AF-{pdb_code}-F1-model_v3.pdb"

In [ ]:
%%bash
wget -qnc https://raw.githubusercontent.com/casperg92/MaSIF_colab/main/example/monomerexample.pdb

In [ ]:
clear_mem()
model = mk_afdesign_model(protocol="binder")

# pos = define positions (hotspot) on target you want to target for binder
model.prep_inputs(pdb_filename="monomerexample.pdb", chain="A",
                  binder_len=20,
                  hotspot="33,17,6,34,30,19,37,15") 

print("target_length",model._target_len)
print("binder_length",model._binder_len)
print("weights",model.opt["weights"])

In [ ]:
model.restart(mode="soft_gumbel")
model.design_3stage(100, 100, 10)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()

In [ ]:
model.plot_pdb()
model.save_pdb(f"{model.protocol}.pdb")

as a control, let's try hallucinate a binder without defining a hotspot

In [ ]:
clear_mem()
model = mk_afdesign_model(protocol="binder")
model.prep_inputs(pdb_filename="monomerexample.pdb", chain="A", binder_len=20)

In [ ]:
model.restart(mode="soft_gumbel")
model.design_3stage(100, 100, 10)

In [ ]:
HTML(model.animate())

In [ ]:
model.get_seqs()

In [ ]:
model.plot_pdb()
model.save_pdb(f"{model.protocol}.control.pdb")